# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

import os
import requests

# Base URL and target folder
base_url = "https://www.gutenberg.org/files/21144/mp3/21144-{:02d}.mp3"
target_folder = "downloads"
file_numbers = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Create target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Download loop
for num in file_numbers:
    file_url = base_url.format(num)
    filename = f"21144-{num:02d}.mp3"
    file_path = os.path.join(target_folder, filename)

    if os.path.exists(file_path):
        print(f"Already exists: {filename}")
        continue

    print(f"Downloading: {filename}")
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {filename}: {e}")


Already exists: 21144-01.mp3
Already exists: 21144-04.mp3
Already exists: 21144-05.mp3
Already exists: 21144-06.mp3
Already exists: 21144-14.mp3
Already exists: 21144-22.mp3
Already exists: 21144-24.mp3
Already exists: 21144-25.mp3
Already exists: 21144-26.mp3
Already exists: 21144-27.mp3


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [5]:
import os
import json
import openai
from dotenv import load_dotenv

# Load environment variables from a `.env` file (e.g., OPENAI_API_KEY=sk-...)
load_dotenv()

# Create the OpenAI client using the API key loaded from environment
client = openai.OpenAI()

# Folder where the MP3 files are stored
AUDIO_FOLDER = "downloads"

# Path to the cache file that stores transcripts to avoid reprocessing
CACHE_FILE = "transcripts_api.json"

# List of file numbers we want to transcribe (e.g., 21144-01.mp3, 21144-04.mp3, etc.)
FILE_NUMBERS = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Load the transcript cache if it already exists, otherwise start with an empty dict
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r", encoding="utf-8") as f:
        transcript_cache = json.load(f)
else:
    transcript_cache = {}

# Function to transcribe a single MP3 file using the OpenAI Whisper-1 API
def transcribe_with_openai(mp3_path):
    # If the transcription is already cached, return it to avoid extra API cost
    if mp3_path in transcript_cache:
        print(f"🗃️ Transcript found in cache: {mp3_path}")
        return transcript_cache[mp3_path]

    print(f"🎙️ Transcribing with Whisper-1: {mp3_path}")
    try:
        # Open the audio file in binary mode
        with open(mp3_path, "rb") as audio_file:
            # Send the file to OpenAI's Whisper-1 transcription API
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
            # Extract the transcript text from the API response
            text = transcript.text

            # Save the result in the cache
            transcript_cache[mp3_path] = text

            # Write the updated cache back to the JSON file
            with open(CACHE_FILE, "w", encoding="utf-8") as f:
                json.dump(transcript_cache, f, ensure_ascii=False, indent=2)

            return text
    except Exception as e:
        print(f"❌ Error transcribing {mp3_path}: {e}")
        return None

# Main loop: go through each file number and transcribe the corresponding MP3
for num in FILE_NUMBERS:
    file_name = f"21144-{num:02d}.mp3"  # Format file name with leading zero (e.g., 01, 04, etc.)
    file_path = os.path.join(AUDIO_FOLDER, file_name)  # Full path to the file

    # Skip files that don't exist
    if not os.path.exists(file_path):
        print(f"⚠️ File not found: {file_path}")
        continue

    # Transcribe the file and print a short summary
    result_text = transcribe_with_openai(file_path)
    if result_text:
        print(f"✅ Transcribed ({file_name}): {len(result_text.split())} palabras")


🎙️ Transcribing with Whisper-1: downloads/21144-01.mp3
✅ Transcribed (21144-01.mp3): 103 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-04.mp3
✅ Transcribed (21144-04.mp3): 142 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-05.mp3
✅ Transcribed (21144-05.mp3): 135 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-06.mp3
✅ Transcribed (21144-06.mp3): 152 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-14.mp3
✅ Transcribed (21144-14.mp3): 103 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-22.mp3
✅ Transcribed (21144-22.mp3): 103 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-24.mp3
✅ Transcribed (21144-24.mp3): 140 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-25.mp3
✅ Transcribed (21144-25.mp3): 92 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-26.mp3
✅ Transcribed (21144-26.mp3): 118 palabras
🎙️ Transcribing with Whisper-1: downloads/21144-27.mp3
✅ Transcribed (21144-27.mp3): 95 palabras


In [6]:
transcript_cache

{'downloads/21144-01.mp3': 'Las fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula Esta es una grabación del dominio público.',
 'downloads/21144-04.mp3': 'Las fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz, fábula número 64, El Lobo y la Cruz. A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correra a una grulla y le pidió que le salvara de aquella situación y que enseguida le pagaría por ello. Aceptó la g

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 4:**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**